# R workshop: <font color=blue> Generating KEGG Pathway view </font>

#### MLBI@DKU

### __0. Install required R packages__

In [123]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
# BiocManager::install("biocLite")
BiocManager::install()

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Old packages: 'data.table', 'purrr', 'rprojroot', 'waldo'



In [124]:
BiocManager::install("org.Hs.eg.db")
BiocManager::install("org.Mm.eg.db")
BiocManager::install("biomaRt")
BiocManager::install("gageData")
BiocManager::install("gage")
BiocManager::install("pathview")
install.packages("filesstrings")
install.packages("anndata")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'org.Hs.eg.db'”
Old packages: 'data.table', 'purrr', 'rprojroot', 'waldo'

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'org.Mm.eg.db'”
Old packages: 'data.table', 'purrr', 'rprojroot', 'waldo'

'getOption("repos")' replaces Bioconductor standard repositories, 

In [125]:
devtools::install_github("combio-dku/KEGGPathviewGen4SCODA")
system('pip install mlbi-datasets --upgrade')
# reticulate::py_install("mlbi-datasets", method='pip')

### __1. Load libraries and data__

In [126]:
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(filesstrings))
suppressPackageStartupMessages(library(pathview))
suppressPackageStartupMessages(library(gage))
suppressPackageStartupMessages(library(gageData))
suppressPackageStartupMessages(library(org.Hs.eg.db))
suppressPackageStartupMessages(library(org.Mm.eg.db))
suppressPackageStartupMessages(library(reticulate))
suppressPackageStartupMessages(library(anndata))
suppressPackageStartupMessages(library(KEGGPathviewGen4SCODA))

reticulate::py_require("gdown")
reticulate::py_require("mlbi-datasets")
mlbi <- reticulate::import("mlbi_datasets")

### __2A. Load your scoda-processed data__

In [ ]:
data.dir <- "./"

flst <- list.files(data.dir)
flst

[1] "BRCA_428K_GSE161529_33K_v1.h5ad"                            
[2] "sample_data"                                                
[3] "scoda_workshop_example_dataset_GSE161529_33K_results.tar.gz"

In [ ]:
file <- 'scoda_workshop_example_dataset_GSE161529_33K_results.tar.gz'
untar(file)

In [ ]:
### Load data & extract cell-gene matrix as a data.frame (rownames: cell barcode, colnames: gene symbol)
file_h5ad <- 'scoda_workshop_example_dataset_GSE161529_33K.h5ad'

adata_t <- read_h5ad(file_h5ad)
adata_t

AnnData object with n_obs × n_vars = 33785 × 22621
    obs: 'Patient', 'Description', 'Source', 'Condition', 'Menopause', 'Parity', 'Gender', 'geo_no', 'sid', 'subtype', 'subtype_detail', 'tissue', 'condition', 'sample', 'sample_rev', 'sample_ext', 'celltype_major', 'celltype_minor', 'celltype_subset', 'cnv_ref_ind', 'ploidy_score', 'ploidy_dec', 'condition_for_deg', 'sample_ext_for_deg', 'celltype_for_deg', 'celltype_for_cci', 'tumor_origin_ind'
    var: 'variable_genes', 'chr', 'spot_no'
    uns: 'CCI', 'CCI_sample', 'Celltype_marker_DB', 'DEG', 'DEG_grouping_vars', 'DEG_stat', 'DEG_vs_ref', 'DEG_vs_ref_stat', 'GSA_down', 'GSA_up', 'GSA_vs_ref_down', 'GSA_vs_ref_up', 'GSEA', 'GSEA_vs_ref', 'Pathways_DB', 'analysis_parameters', 'cnv', 'cnv_neighbors_info', 'inferploidy_summary', 'log', 'lut_sample_to_cond', 'usr_param'
    obsm: 'HiCAT_result', 'X_cnv', 'X_cnv_pca', 'X_pca', 'inferploidy_results'
    obsp: 'cnv_neighbor_graph_connectivity', 'cnv_neighbor_graph_distance'

### __2B. Or, download a preprocessed data__

In [127]:
adata_t <- mlbi$load_scoda_processed_sample_data( 'Breast' )
adata_t

AnnData object with n_obs × n_vars = 33742 × 22585
    obs: 'Patient', 'Description', 'Source', 'Condition', 'Menopause', 'Parity', 'Gender', 'geo_no', 'sid', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'subtype', 'subtype_detail', 'tissue', 'condition', 'sample', 'copykat_dec', 'scevan_dec', 'sample_ext', 'celltype_major', 'celltype_minor', 'celltype_subset', 'cnv_ref_ind', 'ploidy_score', 'ploidy_dec', 'ploidy_init_group', 'condition_for_deg', 'sample_ext_for_deg', 'celltype_for_deg', 'celltype_for_cci', 'tumor_origin_ind'
    var: 'variable_genes', 'chr', 'spot_no'
    uns: 'CCI', 'CCI_sample', 'Celltype_marker_DB', 'DEG', 'DEG_grouping_vars', 'DEG_stat', 'DEG_vs_ref', 'DEG_vs_ref_stat', 'GSA_down', 'GSA_up', 'GSA_vs_ref_down', 'GSA_vs_ref_up', 'GSEA', 'GSEA_vs_ref', 'Pathways_DB', 'analysis_parameters', 'cnv', 'cnv_neighbors_info', 'inferploidy_summary', 'log', 'lut_sample_to_cond', 'usr_param'
    obsm: 'HiCAT_result', 'X_cnv', 'X_cnv_pca', 'X_pca', 'i

### __3. Get KEGG pathway IDs and names__

In [128]:
species <- adata_t$uns[['usr_param']][['species']]
pathways.used <- adata_t$uns[['Pathways_DB']]

df_pathways_map <- get_pathways_map( pathways.used, species, min_overlap = 0.85 )

  |======================================================================| 100%
Converting Pathways DB .. done.        


### __4. Generate KEGG pathview__

In [129]:
lst.deg.all <- adata_t$uns[['DEG']]
lst.gsa.all <- adata_t$uns[['GSA_up']]

lst.fcs.all <- get_all_fold_changes( lst.deg.all, species )

Getting fold changes .. 
     Aneuploid Epithelial cell: ER+_vs_others(3656), HER2+_vs_others(3061), TNBC_vs_others(5190)
                        B cell: ER+_vs_others(38), HER2+_vs_others(31), TNBC_vs_others(46)
       Diploid Epithelial cell: ER+_vs_others(805), HER2+_vs_others(880), Normal_vs_others(3754), TNBC_vs_others(1116)
              Endothelial cell: ER+_vs_others(93), HER2+_vs_others(104), Normal_vs_others(2645), TNBC_vs_others(299)
               Epithelial cell: Diploid_vs_others(2440), ER+_vs_others(2278), HER2+_vs_others(1751), Normal_vs_others(3221), TNBC_vs_others(3020)
                    Fibroblast: ER+_vs_others(298), HER2+_vs_others(236), Normal_vs_others(4011), TNBC_vs_others(775)
                           ILC: ER+_vs_others(55), HER2+_vs_others(35), Normal_vs_others(479), TNBC_vs_others(38)
                    Macrophage: ER+_vs_others(260), HER2+_vs_others(188), Normal_vs_others(2145), TNBC_vs_others(609)
                     Mast cell: ER+_vs_others(60)
     

In [138]:
target_cell <- 'Epithelial cell'
dir_saved <- save_kegg_pathviews( target_cell, lst.gsa.all,
                                  lst.fcs.all, df_pathways_map,
                                  species, gsa.p.val.cutoff = 1e-2 )

  |======================================================================| 100%
               Epithelial cell: 5/5 - 7/7 - Thermogenesis             


In [ ]:
names(lst.fcs.all)

[1] "Aneuploid Epithelial cell" "B cell"                   
 [3] "Diploid Epithelial cell"   "Endothelial cell"         
 [5] "Epithelial cell"           "Fibroblast"               
 [7] "ILC"                       "Macrophage"               
 [9] "Mast cell"                 "Plasma cell"              
[11] "Smooth muscle cell"        "T cell CD4+"              
[13] "T cell CD8+"

In [ ]:
for( target_cell in names(lst.fcs.all) )
{
    dir_saved <- save_kegg_pathviews( target_cell, lst.gsa.all, lst.fcs.all, df_pathways_map, species, p.val.cutoff = 0.01 )
}